In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import tarfile
import cv2
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import time
import matplotlib.patches as patches
from align import AlignDlib
from keras.models import model_from_json
from keras.models import load_model
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score 

Using TensorFlow backend.


In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
ATTRS_NAME = "./data/lfw_attributes.txt"
IMAGES_NAME = "./data/lfw-deepfunneled.tgz"
RAW_IMAGES_NAME = "./data/lfw.tgz"

In [3]:
def decoder(raw_bytes):
    img = cv2.imdecode(np.asarray(bytearray(raw_bytes), dtype=np.uint8), 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [4]:
def load_dataset(path):
    img_data_list=[]
    data_path = path

    data_dir_list = os.listdir(data_path)
    for dataset in data_dir_list:
        img_list=os.listdir(data_path+'/'+ dataset)
        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
        for img in img_list:
            input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
            input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
            #input_img_resize=cv2.resize(input_img,(64,64))
            faces = face_cascade.detectMultiScale(input_img, 1.1, 3)
            print (faces[0])
            #input_img_resize=cv2.resize(input_img,(128,128))
            x,y,w,h = faces[0]
            print(x,y,w,h)
            face = input_img[y:y+h,x:x+w]
            face=cv2.resize(face,(128,128))
            #face=face / 255.0 - 0.5
            #face = np.expand_dims(faces, axis=1)
            img_data_list.append(face)

    img_data = np.stack(img_data_list).astype('uint8')
    #img_data = img_data.astype('float32')
    return img_data/ 255.0 - 0.5

In [5]:
def load_lfw_dataset(
        use_raw=False,
        dx=80, dy=80,
        dimx=45, dimy=45):

    # Read attrs
    df_attrs = pd.read_csv(ATTRS_NAME, sep='\t', skiprows=1)
    df_attrs = pd.DataFrame(df_attrs.iloc[:, :-1].values, columns=df_attrs.columns[1:])
    df_attrs.head()
    imgs_with_attrs = set(map(tuple, df_attrs[["person", "imagenum"]].values))

    # Read photos
    all_photos = []
    photo_ids = []


    # Read attrs
    #df_attrs = pd.read_csv(ATTRS_NAME, sep='\t', skiprows=1)
    #df_attrs = pd.DataFrame(df_attrs.iloc[:, :-1].values, columns=df_attrs.columns[1:])
    #df_attrs.head()
    with tarfile.open(RAW_IMAGES_NAME if use_raw else IMAGES_NAME) as f:
        for m in f.getmembers():
            # Only process image files from the compressed data
            print(m)
            if m.isfile() and m.name.endswith(".jpg"):
                # Prepare image
                img = decoder(f.extractfile(m).read())

                # Crop only faces and resize it
                img = img[dy:-dy, dx:-dx]
                img = cv2.resize(img, (dimx, dimy))

                # Parse person and append it to the collected data
                foldername = os.path.split(m.name)
                filename = os.path.split(m.name)[-1]
                fname_splitted = filename[:-4].replace('_', ' ').split()
                person_id = ' '.join(fname_splitted[:-1])
                photo_number = int(fname_splitted[-1])
                if (person_id, photo_number) in imgs_with_attrs:
                    all_photos.append(img)
                    photo_ids.append({'Person': person_id, 'imagenum': photo_number, 'Array': img, "File": filename, "Path": foldername })

    photo_ids = pd.DataFrame(photo_ids)
    all_photos = np.stack(all_photos).astype('uint8')
    all_photos = all_photos.astype('float32') / 255.0 - 0.5

    # Preserve photo_ids order!
    #all_attrs = photo_ids.merge(df_attrs, on=('person', 'imagenum'))

    return all_photos, photo_ids#all_attrs

In [6]:
X, photosDf = load_lfw_dataset(use_raw=True, dimx=128, dimy=128)

<TarInfo 'lfw' at 0x7efebffb6460>
<TarInfo 'lfw/George_HW_Bush' at 0x7efebffb6390>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0001.jpg' at 0x7efebffb61f0>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0002.jpg' at 0x7efebffb6530>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0003.jpg' at 0x7efebffb6600>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0004.jpg' at 0x7efebffb67a0>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0005.jpg' at 0x7efebffb6870>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0006.jpg' at 0x7efebffb6940>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0007.jpg' at 0x7efebffb6a10>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0008.jpg' at 0x7efebffb6ae0>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0009.jpg' at 0x7efebffb6bb0>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0010.jpg' at 0x7efebffb6c80>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0011.jpg' at 0x7efebffb6d50>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bush_0012.jpg' at 0x7efebffb6e20>
<TarInfo 'lfw/George_HW_Bush/George_HW_Bu

<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0008.jpg' at 0x7efebf967390>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0009.jpg' at 0x7efebf967460>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0010.jpg' at 0x7efebf967530>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0011.jpg' at 0x7efebf967600>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0012.jpg' at 0x7efebf9676d0>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0013.jpg' at 0x7efebf9677a0>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0014.jpg' at 0x7efebf967870>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0015.jpg' at 0x7efebf967940>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0016.jpg' at 0x7efebf967a10>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0017.jpg' at 0x7efebf967ae0>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0018.jpg' at 0x7efebf967bb0>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_0019.jpg' at 0x7efebf967c80>
<TarInfo 'lfw/Gerhard_Schroeder/Gerhard_Schroeder_00

<TarInfo 'lfw/David_Caruso' at 0x7efebf91a940>
<TarInfo 'lfw/David_Caruso/David_Caruso_0001.jpg' at 0x7efebf91ad50>
<TarInfo 'lfw/David_Caruso/David_Caruso_0002.jpg' at 0x7efebf91ac80>
<TarInfo 'lfw/David_Caruso/David_Caruso_0003.jpg' at 0x7efebf91aef0>
<TarInfo 'lfw/Edmund_Stoiber' at 0x7efebf91ae20>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0001.jpg' at 0x7efebf91e120>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0002.jpg' at 0x7efebf91e050>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0003.jpg' at 0x7efebf91e2c0>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0004.jpg' at 0x7efebf91e390>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0005.jpg' at 0x7efebf91e460>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0006.jpg' at 0x7efebf91e530>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0007.jpg' at 0x7efebf91e600>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0008.jpg' at 0x7efebf91e6d0>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoiber_0009.jpg' at 0x7efebf91e7a0>
<TarInfo 'lfw/Edmund_Stoiber/Edmund_Stoi

<TarInfo 'lfw/George_W_Bush/George_W_Bush_0207.jpg' at 0x7efebf94dc80>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0208.jpg' at 0x7efebf94dd50>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0209.jpg' at 0x7efebf94de20>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0210.jpg' at 0x7efebf94def0>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0211.jpg' at 0x7efebf94d050>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0212.jpg' at 0x7efebf8d2050>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0213.jpg' at 0x7efebf8d2120>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0214.jpg' at 0x7efebf8d22c0>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0215.jpg' at 0x7efebf8d2390>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0216.jpg' at 0x7efebf8d2460>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0217.jpg' at 0x7efebf8d2530>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0218.jpg' at 0x7efebf8d2600>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0219.jpg' at 0x7efebf8d26d0>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0220.jpg' at 0x7efebf8d27a0>
<TarIn

<TarInfo 'lfw/George_W_Bush/George_W_Bush_0386.jpg' at 0x7efebf8f7390>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0387.jpg' at 0x7efebf8f7460>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0388.jpg' at 0x7efebf8f7530>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0389.jpg' at 0x7efebf8f7600>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0390.jpg' at 0x7efebf8f76d0>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0391.jpg' at 0x7efebf8f77a0>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0392.jpg' at 0x7efebf8f7870>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0393.jpg' at 0x7efebf8f7940>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0394.jpg' at 0x7efebf8f7a10>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0395.jpg' at 0x7efebf8f7ae0>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0396.jpg' at 0x7efebf8f7bb0>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0397.jpg' at 0x7efebf8f7c80>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0398.jpg' at 0x7efebf8f7d50>
<TarInfo 'lfw/George_W_Bush/George_W_Bush_0399.jpg' at 0x7efebf8f7e20>
<TarIn

...

<TarInfo 'lfw/James_Sensenbrenner' at 0x7efebea78940>
<TarInfo 'lfw/James_Sensenbrenner/James_Sensenbrenner_0001.jpg' at 0x7efebea78c80>
<TarInfo 'lfw/William_McDonough' at 0x7efebea78bb0>
<TarInfo 'lfw/William_McDonough/William_McDonough_0001.jpg' at 0x7efebea78e20>
<TarInfo 'lfw/Danny_Glover' at 0x7efebea78d50>
<TarInfo 'lfw/Danny_Glover/Danny_Glover_0001.jpg' at 0x7efebea78ef0>
<TarInfo 'lfw/Jimmy_Lee' at 0x7efebea7c050>
<TarInfo 'lfw/Jimmy_Lee/Jimmy_Lee_0001.jpg' at 0x7efebea7c1f0>
<TarInfo 'lfw/Peter_Fitzgerald' at 0x7efebea7c120>
<TarInfo 'lfw/Peter_Fitzgerald/Peter_Fitzgerald_0001.jpg' at 0x7efebea7c390>
<TarInfo 'lfw/Kirk_Franklin' at 0x7efebea7c2c0>
<TarInfo 'lfw/Kirk_Franklin/Kirk_Franklin_0001.jpg' at 0x7efebea7c530>
<TarInfo 'lfw/Eduardo_Romero' at 0x7efebea7c460>
<TarInfo 'lfw/Eduardo_Romero/Eduardo_Romero_0001.jpg' at 0x7efebea7c6d0>
<TarInfo 'lfw/Dennis_Oswald' at 0x7efebea7c600>
<TarInfo 'lfw/Dennis_Oswald/Dennis_Oswald_0001.jpg' at 0x7efebea7c870>
<TarInfo 'lfw/Justin_

<TarInfo 'lfw/Queen_Silvia' at 0x7efebea4b120>
<TarInfo 'lfw/Queen_Silvia/Queen_Silvia_0001.jpg' at 0x7efebea4b390>
<TarInfo 'lfw/Dan_LaCoutre' at 0x7efebea4b2c0>
<TarInfo 'lfw/Dan_LaCoutre/Dan_LaCoutre_0001.jpg' at 0x7efebea4b530>
<TarInfo 'lfw/Eminem' at 0x7efebea4b460>
<TarInfo 'lfw/Eminem/Eminem_0001.jpg' at 0x7efebea4b6d0>
<TarInfo 'lfw/George_Blaney' at 0x7efebea4b600>
<TarInfo 'lfw/George_Blaney/George_Blaney_0001.jpg' at 0x7efebea4b870>
<TarInfo 'lfw/Michael_Winterbottom' at 0x7efebea4b7a0>
<TarInfo 'lfw/Michael_Winterbottom/Michael_Winterbottom_0001.jpg' at 0x7efebea4ba10>
<TarInfo 'lfw/Michael_Winterbottom/Michael_Winterbottom_0002.jpg' at 0x7efebea4b940>
<TarInfo 'lfw/Michael_Winterbottom/Michael_Winterbottom_0003.jpg' at 0x7efebea4bbb0>
<TarInfo 'lfw/Albert_Montanes' at 0x7efebea4bc80>
<TarInfo 'lfw/Albert_Montanes/Albert_Montanes_0001.jpg' at 0x7efebea4bd50>
<TarInfo 'lfw/Daniel_Osorno' at 0x7efebea4bae0>
<TarInfo 'lfw/Daniel_Osorno/Daniel_Osorno_0001.jpg' at 0x7efebea4bef

<TarInfo 'lfw/Annika_Sorenstam' at 0x7efebe99c530>
<TarInfo 'lfw/Annika_Sorenstam/Annika_Sorenstam_0001.jpg' at 0x7efebe99c7a0>
<TarInfo 'lfw/Mohamed_Seineldin' at 0x7efebe99c6d0>
<TarInfo 'lfw/Mohamed_Seineldin/Mohamed_Seineldin_0001.jpg' at 0x7efebe99c940>
<TarInfo 'lfw/Kai-Uwe_Ricke' at 0x7efebe99c870>
<TarInfo 'lfw/Kai-Uwe_Ricke/Kai-Uwe_Ricke_0001.jpg' at 0x7efebe99cae0>
<TarInfo 'lfw/Nida_Blanca' at 0x7efebe99ca10>
<TarInfo 'lfw/Nida_Blanca/Nida_Blanca_0001.jpg' at 0x7efebe99cc80>
<TarInfo 'lfw/Robbie_Naish' at 0x7efebe99cbb0>
<TarInfo 'lfw/Robbie_Naish/Robbie_Naish_0001.jpg' at 0x7efebe99ce20>
<TarInfo 'lfw/Scott_Weiland' at 0x7efebe99cd50>
<TarInfo 'lfw/Scott_Weiland/Scott_Weiland_0001.jpg' at 0x7efebe99cef0>
<TarInfo 'lfw/Patrice_Chereau' at 0x7efebe9a1050>
<TarInfo 'lfw/Patrice_Chereau/Patrice_Chereau_0001.jpg' at 0x7efebe9a11f0>
<TarInfo 'lfw/Patrice_Chereau/Patrice_Chereau_0002.jpg' at 0x7efebe9a1120>
<TarInfo 'lfw/Michael_Haneke' at 0x7efebe9a1390>
<TarInfo 'lfw/Michael_Han

<TarInfo 'lfw/Takeo_Hiranuma' at 0x7efebe9711f0>
<TarInfo 'lfw/Takeo_Hiranuma/Takeo_Hiranuma_0001.jpg' at 0x7efebe971460>
<TarInfo 'lfw/Mark_Salter' at 0x7efebe971390>
<TarInfo 'lfw/Mark_Salter/Mark_Salter_0001.jpg' at 0x7efebe971600>
<TarInfo 'lfw/Charles_Lebois' at 0x7efebe971530>
<TarInfo 'lfw/Charles_Lebois/Charles_Lebois_0001.jpg' at 0x7efebe9717a0>
<TarInfo 'lfw/Keith_Urban' at 0x7efebe9716d0>
<TarInfo 'lfw/Keith_Urban/Keith_Urban_0001.jpg' at 0x7efebe971940>
<TarInfo 'lfw/Sedigh_Barmak' at 0x7efebe971870>
<TarInfo 'lfw/Sedigh_Barmak/Sedigh_Barmak_0001.jpg' at 0x7efebe971ae0>
<TarInfo 'lfw/Theresa_Gattung' at 0x7efebe971a10>
<TarInfo 'lfw/Theresa_Gattung/Theresa_Gattung_0001.jpg' at 0x7efebe971c80>
<TarInfo 'lfw/Dean_Sheremet' at 0x7efebe971bb0>
<TarInfo 'lfw/Dean_Sheremet/Dean_Sheremet_0001.jpg' at 0x7efebe971e20>
<TarInfo 'lfw/Choi_Yun-yong' at 0x7efebe971d50>
<TarInfo 'lfw/Choi_Yun-yong/Choi_Yun-yong_0001.jpg' at 0x7efebe971ef0>
<TarInfo 'lfw/Bruce_Arena' at 0x7efebe972050>
<T

<TarInfo 'lfw/Leticia_Dolera' at 0x7efebe944bb0>
<TarInfo 'lfw/Leticia_Dolera/Leticia_Dolera_0001.jpg' at 0x7efebe944ef0>
<TarInfo 'lfw/Joseph_Fiennes' at 0x7efebe944e20>
<TarInfo 'lfw/Joseph_Fiennes/Joseph_Fiennes_0001.jpg' at 0x7efebe948120>
<TarInfo 'lfw/Colin_Prescot' at 0x7efebe948050>
<TarInfo 'lfw/Colin_Prescot/Colin_Prescot_0001.jpg' at 0x7efebe9482c0>


In [7]:
#X = load_dataset("../data/fotop/")

In [8]:
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model

def build_encoder(img_shape, code_size):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(img_shape))
    encoder.add(Flatten())
    encoder.add(Dense(code_size))
    return encoder

In [9]:
encodetor = build_encoder((128, 128, 3), 1000)
encodetor.load_weights('./data/models/encoderweights128.h5')
encodetor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 49152)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              49153000  
Total params: 49,153,000
Trainable params: 49,153,000
Non-trainable params: 0
_________________________________________________________________


In [10]:
vectors = encodetor.predict(X)

In [11]:
photosDf

,Person,imagenum,Array,File,Path
0,George HW Bush,1,"[[[144, 123, 104], [144, 126, 106], [144, 127,...",George_HW_Bush_0001.jpg,"(lfw/George_HW_Bush, George_HW_Bush_0001.jpg)"
1,George HW Bush,3,"[[[171, 100, 38], [175, 102, 38], [179, 104, 3...",George_HW_Bush_0003.jpg,"(lfw/George_HW_Bush, George_HW_Bush_0003.jpg)"
2,George HW Bush,4,"[[[180, 180, 144], [179, 181, 144], [178, 180,...",George_HW_Bush_0004.jpg,"(lfw/George_HW_Bush, George_HW_Bush_0004.jpg)"
3,George HW Bush,5,"[[[154, 132, 118], [158, 134, 120], [163, 137,...",George_HW_Bush_0005.jpg,"(lfw/George_HW_Bush, George_HW_Bush_0005.jpg)"
4,George HW Bush,6,"[[[97, 141, 108], [95, 139, 105], [90, 134, 98...",George_HW_Bush_0006.jpg,"(lfw/George_HW_Bush, George_HW_Bush_0006.jpg)"
...,...,...,...,...,...
13138,Dino de Laurentis,2,"[[[141, 119, 98], [136, 112, 90], [129, 103, 7...",Dino_de_Laurentis_0002.jpg,"(lfw/Dino_de_Laurentis, Dino_de_Laurentis_0002..."
13139,Francis Ricciardone,1,"[[[240, 242, 239], [239, 241, 237], [240, 242,...",Francis_Ricciardone_0001.jpg,"(lfw/Francis_Ricciardone, Francis_Ricciardone_..."
13140,Leticia Dolera,1,"[[[16, 9, 3], [17, 10, 3], [18, 11, 2], [19, 1...",Leticia_Dolera_0001.jpg,"(lfw/Leticia_Dolera, Leticia_Dolera_0001.jpg)"
13141,Joseph Fiennes,1,"[[[214, 214, 222], [214, 214, 223], [214, 214,...",Joseph_Fiennes_0001.jpg,"(lfw/Joseph_Fiennes, Joseph_Fiennes_0001.jpg)"


In [12]:
vectors.shape

(13143, 1000)

In [13]:
names = [f for f in photosDf.Path]
names

[('lfw/George_HW_Bush', 'George_HW_Bush_0001.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0003.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0004.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0005.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0006.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0007.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0008.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0009.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0010.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0011.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0012.jpg'),
 ('lfw/George_HW_Bush', 'George_HW_Bush_0013.jpg'),
 ('lfw/Curtis_Strange', 'Curtis_Strange_0001.jpg'),
 ('lfw/Marc_Grossman', 'Marc_Grossman_0001.jpg'),
 ('lfw/Marc_Grossman', 'Marc_Grossman_0002.jpg'),
 ('lfw/Marc_Grossman', 'Marc_Grossman_0003.jpg'),
 ('lfw/Marc_Grossman', 'Marc_Grossman_0004.jpg'),
 ('lfw/Michael_Schumacher', 'Michael_Schumacher_0001.jpg'),
 ('lfw/Michael_Schumacher', 'Michael_Schumacher_0002.jpg'),
 ('l

In [15]:
def recomender(vectors,names,img_path):
    mod = list(vectors)
    s_N = names
    s = pd.Series(s_N)
    s_vector = ([mod])
    df=pd.DataFrame(s)
    df['Vector']=mod
    df.columns=(['name','X'])
    test_person=load_dataset(img_path)
    test_person_pred=encodetor.predict(test_person)
    X_missing=test_person_pred
    df["diffs"] = df["X"].apply(lambda X: np.linalg.norm(X-X_missing))
    results = df.groupby("name").agg({'diffs':'min'}).sort_values(by='diffs')
    return results.index[0]
res = recomender(vectors,names,'./data/elegido')
res

Loaded the images of dataset-Fran

[195 165 150 150]
195 165 150 150


('lfw/George_W_Bush', 'George_W_Bush_0475.jpg')